**This project is about using neural network to train a sequence to sequence model on a dataset of English and French**
**sentences that can translate  new senetence from English to French**

**Import the packages need, connect to gmail for authentication as colab is used to read the data**

In [0]:
!pip install -U -q PyDrive

In [0]:
import sqlite3
import numpy as np
import pylab as pl
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score

%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [0]:
import keras
import psutil
import pandas as pd
import tensorflow as tf
from google.colab import auth
from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

Using TensorFlow backend.


In [0]:
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, Embedding, RepeatVector, TimeDistributed
import numpy as np

from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
from time import time

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file1 = drive.CreateFile({'id':'1Fs_lfbowZAWrBFV4OAtpQI_S5LeibYdj'})
file1.GetContentFile('Translation_Data.zip')

In [0]:
!ls

adc.json  sample_data  Translation_Data.zip


In [0]:
!unzip -qq Translation_Data.zip  

In [0]:
!ls

adc.json  sample_data	      small_vocab_fr.txt
__MACOSX  small_vocab_en.txt  Translation_Data.zip


**Split the english and french document at new line character so that it can be each sentence**

In [0]:
eng_chars = []
fra_chars = []


eng_sent = open('small_vocab_en.txt', encoding='utf-8').read().split('\n')
fra_sent = open('small_vocab_fr.txt', encoding='utf-8').read().split('\n')

In [0]:
print(len(eng_sent))
print(len(fra_sent))

137861
137861


In [0]:
print(eng_sent[0])
print(fra_sent[0])
			



new jersey is sometimes quiet during autumn , and it is snowy in april .
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .


In [0]:
print(eng_sent[1])
print(fra_sent[1])

the united states is usually chilly during july , and it is usually freezing in november .
les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


**Get each line and take the word and add to english and french dictionary**

In [0]:
nb_samples = len(fra_sent)

for line in range(nb_samples):
  eng_line = eng_sent[line]
  fra_line = fra_sent[line]
  
  
    

  for ch in eng_line:
        if (ch not in eng_chars):
            eng_chars.append(ch)
  for ch in fra_line:
        if (ch not in fra_chars):
            fra_chars.append(ch)
        
#fra_chars = sorted(list(fra_chars))
#eng_chars = sorted(list(eng_chars))



In [0]:
print(eng_sent[0])
print(fra_sent[0])

new jersey is sometimes quiet during autumn , and it is snowy in april .
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .


1) **create a dictionary to index each english character - key is index and value is english character**
2)** create a dictionary to get english character given its index - key is english character and value is index**
3) **create a dictionary to index each french character - key is index and value is french character**
4)** create a dictionary to get french character given its index - key is french character and value is index**

In [0]:
# dictionary to index each english character - key is index and value is english character
eng_index_to_char_dict = {}

# dictionary to get english character given its index - key is english character and value is index
eng_char_to_index_dict = {}

for k, v in enumerate(eng_chars):
    eng_index_to_char_dict[k] = v
    eng_char_to_index_dict[v] = k

# dictionary to index each french character - key is index and value is french character
fra_index_to_char_dict = {}

# dictionary to get french character given its index - key is french character and value is index
fra_char_to_index_dict = {}
for k, v in enumerate(fra_chars):
    fra_index_to_char_dict[k] = v
    fra_char_to_index_dict[v] = k

max_len_eng_sent = max([len(line) for line in eng_sent])
max_len_fra_sent = max([len(line) for line in fra_sent])


In [0]:
print(max_len_eng_sent)
print(max_len_fra_sent)

102
114


**Create a one hot encoding of Frecnch and English senetences**

In [0]:
tokenized_eng_sentences = np.zeros(shape=(nb_samples, max_len_eng_sent, len(eng_chars)), dtype='float32')
tokenized_fra_sentences = np.zeros(shape=(nb_samples, max_len_fra_sent, len(fra_chars)), dtype='float32')
target_data = np.zeros((nb_samples, max_len_fra_sent, len(fra_chars)), dtype='float32')

# Vectorize the english and french sentences

for i in range(nb_samples):
    for k, ch in enumerate(eng_sent[i]):
        tokenized_eng_sentences[i, k, eng_char_to_index_dict[ch]] = 1

    for k, ch in enumerate(fra_sent[i]):
        tokenized_fra_sentences[i, k, fra_char_to_index_dict[ch]] = 1

        # decoder_target_data will be ahead by one timestep and will not include the start character.
        if k > 0:
            target_data[i, k - 1, fra_char_to_index_dict[ch]] = 1

**Create a Encode for the english  word and Decode for the french word**

In [0]:
# Encoder model
encoder_input = Input(shape=(None, len(eng_chars)))
encoder_LSTM = LSTM(256, return_state=True)
encoder_outputs, encoder_h, encoder_c = encoder_LSTM(encoder_input)
encoder_states = [encoder_h, encoder_c]

# Decoder model
decoder_input = Input(shape=(None, len(fra_chars)))
decoder_LSTM = LSTM(256, return_sequences=True, return_state=True)
decoder_out, _, _ = decoder_LSTM(decoder_input, initial_state=encoder_states)
decoder_dense = Dense(len(fra_chars), activation='softmax')
decoder_out = decoder_dense(decoder_out)


**Run the model**

In [0]:
model = Model(inputs=[encoder_input, decoder_input], outputs=[decoder_out])

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(x=[tokenized_eng_sentences, tokenized_fra_sentences],
          y=target_data,
          batch_size=64,
          epochs=40,
          validation_split=0.2, verbose=1, callbacks=[tensorboard])

Train on 110288 samples, validate on 27573 samples
Epoch 1/40
110288/110288 [==============================] - 721s 7ms/step - loss: 0.3996 - val_loss: 0.1486
Epoch 2/40
110288/110288 [==============================] - 715s 6ms/step - loss: 0.1204 - val_loss: 0.1145
Epoch 3/40
110288/110288 [==============================] - 714s 6ms/step - loss: 0.1062 - val_loss: 0.1101
Epoch 4/40
110288/110288 [==============================] - 686s 6ms/step - loss: 0.0984 - val_loss: 0.1007
Epoch 5/40
110288/110288 [==============================] - 693s 6ms/step - loss: 0.0933 - val_loss: 0.0936
Epoch 6/40
110288/110288 [==============================] - 686s 6ms/step - loss: 0.0887 - val_loss: 0.0871
Epoch 7/40
110288/110288 [==============================] - 683s 6ms/step - loss: 0.0845 - val_loss: 0.0868
Epoch 8/40
110288/110288 [==============================] - 682s 6ms/step - loss: 0.0807 - val_loss: 0.0832
Epoch 9/40
110288/110288 [==============================] - 699s 6ms/step - loss: 0.0

**Create an interface to take the values from the encode to feed into the decode**

In [0]:
# Inference models for testing

# Encoder inference model
encoder_model_inf = Model(encoder_input, encoder_states)

# Decoder inference model
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

decoder_out, decoder_h, decoder_c = decoder_LSTM(decoder_input,
                                                 initial_state=decoder_input_states)

decoder_states = [decoder_h , decoder_c]

decoder_out = decoder_dense(decoder_out)

decoder_model_inf = Model(inputs=[decoder_input] + decoder_input_states,
                          outputs=[decoder_out] + decoder_states )


**Take each work of English and translate into French**

In [0]:
def decode_seq(inp_seq):
    # Initial states value is coming from the encoder
    states_val = encoder_model_inf.predict(inp_seq)

    target_seq = np.zeros((1, 1, len(fra_chars)))
   # target_seq[0, 0, fra_char_to_index_dict['\t']] = 1

    translated_sent = ''
    stop_condition = False

    while not stop_condition:

        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)

        max_val_index = np.argmax(decoder_out[0, -1, :])
        sampled_fra_char = fra_index_to_char_dict[max_val_index]
        translated_sent += sampled_fra_char

        if ((sampled_fra_char == '\n') or (len(translated_sent) > max_len_fra_sent)):
            stop_condition = True

        target_seq = np.zeros((1, 1, len(fra_chars)))
        target_seq[0, 0, max_val_index] = 1

        states_val = [decoder_h, decoder_c]

    return translated_sent


In [0]:
for seq_index in range(10):
    inp_seq = tokenized_eng_sentences[seq_index:seq_index+1]
    translated_sent = decode_seq(inp_seq)
    print('-')
    print('Input sentence:', eng_sent[seq_index])
    print('Decoded sentence:', translated_sent)

-
Input sentence: new jersey is sometimes quiet during autumn , and it is snowy in april .
Decoded sentence: anjeralemest le fruit merns lamplemons , mais la poire elle est moins aimé...mais .oine .'oir .a nour .....ét..s .a
-
Input sentence: the united states is usually chilly during july , and it is usually freezing in november .
Decoded sentence: es états-unis est généralement froid en juillet , et il est jamais chaud en avril .t .ais il est parfois froid en j
-
Input sentence: california is usually quiet during march , and it is usually hot in june .
Decoded sentence: a fruit est généralement calme en mars , et il est parfois froid en juin .t .ais il est parfois chaud en juillet ..
-
Input sentence: the united states is sometimes mild during june , and it is cold in september .
Decoded sentence: es - rons est parfois froid en juin , et il est parfois chaud en juillet ..t en été est jamais chaud .a printemps .
-
Input sentence: your least liked fruit is the grape , but my least like